# 04 - Applied ML

## Deadline
Monday November 21, 2016 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution
you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code

## Background
In this homework we will gain experience on Applied Machine Learning, exploring an interesting dataset about soccer players and referees.
You can find all the data in the `CrowdstormingDataJuly1st.csv` file, while you can read a thorough [dataset description here](DATA.md).
Given that the focus of this homework is Machine Learning, I recommend you to first take a look at [this notebook](http://nbviewer.jupyter.org/github/mathewzilla/redcard/blob/master/Crowdstorming_visualisation.ipynb)
containing a solid work in pre-processing + visualization of the given dataset. You are *not* allowed to just copy/paste the pre-processing steps
performed by the notebook authors -- you are still supposed to perform your own data analysis for the homework. Still, I'm confident that consulting first
the work done by expert data analysts will speed up tangibly your effort (i.e., they have already found for you many glitches in the data :)


## Assignment
1. Train a `sklearn.ensemble.RandomForestClassifier` that given a soccer player description outputs his skin color. Show how different parameters 
passed to the Classifier affect the overfitting issue. Perform cross-validation to mitigate the overfitting of your model. Once you assessed your model,
inspect the `feature_importances_` attribute and discuss the obtained results. With different assumptions on the data (e.g., dropping certain features even
before feeding them to the classifier), can you obtain a substantially different `feature_importances_` attribute?
*BONUS*: plot the learning curves against at least 2 different sets of parameters passed to your Random Forest. To obtain smooth curves, partition
your data in at least 20 folds. Can you find a set of parameters that leads to high bias, and one which does not?

2. Aggregate the referee information grouping by soccer player, and use an unsupervised learning technique to cluster the soccer players in 2 disjoint
clusters. Remove features iteratively, and at each step perform again the clustering and compute the silhouette score -- can you find a configuration of features with high silhouette
score where players with dark and light skin colors belong to different clusters? Discuss the obtained results.


--------------------------------------------------------------------------------------------------------------
# Data Description

From a company for sports statistics, we obtained data and profile photos from all soccer players (N = 2053) playing in the first male divisions of England, Germany, France and Spain in the 2012-2013 season and all referees (N = 3147) that these players played under in their professional career. We created a dataset of player–referee dyads including the number of matches players and referees encountered each other and our dependent variable, the number of red cards given to a player by a particular referee throughout all matches the two encountered each other.
 
Player photos were available from the source for 1586 out of 2053 players. Players’ skin tone was coded by two independent raters blind to the research question who, based on their profile photo, categorized players on a 5-point scale ranging from “very light skin” to “very dark skin” with “neither dark nor light skin” as the center value. 

Additionally, implicit bias scores for each referee country were calculated using a race implicit association test (IAT), with higher values corresponding to faster white | good, black | bad associations. Explicit bias scores for each referee country were calculated using a racial thermometer task, with higher values corresponding to greater feelings of warmth toward whites versus blacks. Both these measures were created by aggregating data from many online users in referee countries taking these tests on [Project Implicit](http://projectimplicit.net).

In all, the dataset has a total of 146028 dyads of players and referees. A detailed description of all variables in the dataset can be seen in the list below.

## Variables:

*playerShort* - short player ID

*player* - player name

*club* - player club

*leagueCountry* - country of player club (England, Germany, France, and Spain)

*birthday* - player birthday

*height* - player height (in cm)

*weight* - player weight (in kg)

*position* - detailed player position

*games* - number of games in the player-referee dyad

*victories* - victories in the player-referee dyad

*ties* - ties in the player-referee dyad

*defeats* - losses in the player-referee dyad

*goals* - goals scored by a player in the player-referee dyad

*yellowCards* - number of yellow cards player received from referee

*yellowReds* - number of yellow-red cards player received from referee

*redCards* - number of red cards player received from referee

*photoID* - ID of player photo (if available)

*rater1* - skin rating of photo by rater 1 (5-point scale ranging from “very light skin” to “very dark skin”)

*rater2* - skin rating of photo by rater 2 (5-point scale ranging from “very light skin” to “very dark skin”)

*refNum* - unique referee ID number (referee name removed for anonymizing purposes)

*refCountry* - unique referee country ID number (country name removed for anonymizing purposes)

*meanIAT* - mean implicit bias score (using the race IAT) for referee country, higher values correspond to faster white | good, black | bad associations

*nIAT* - sample size for race IAT in that particular country

*seIAT* - standard error for mean estimate of race IAT

*meanExp* - mean explicit bias score (using a racial thermometer task) for referee country, higher values correspond to greater feelings of warmth toward whites versus blacks

*nExp* - sample size for explicit bias in that particular country

*seExp* - standard error for mean estimate of explicit bias measure




--------------------------------------------------------------------------------------------------------------

# IMPLEMENTATION
## Supervised Classification

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.metrics import metrics
import preprocessing_helper as preproc_helper
import classifiers_helper as class_helper
import aggregater_helper as aggr_helper
from statistics import mode

In [ ]:
data = 'CrowdstormingDataJuly1st.csv'
dyads = pd.read_csv(data)

### 1. Visualization of the data

In [ ]:
dyads.ix[:5, :int(len(dyads.columns) / 2)]

In [ ]:
dyads.ix[:5, 14:len(dyads.columns)]

Through the visualization of the data and the ipython notebook which previously worked on the same dataset, we observed that each data entry is a dyad between a player and a referee. They all contain a player ID, a referee ID, their respective descriptors and some informations about the dyad itself.

We could define a dyad as a relationship between a player and a referee. A dyad can contain multiple encounters of the pair (player <-> referee) from different matches. 



### 2. Cleaning 

First of all, as we have (player <-> referee) dyads, a referee in our dataset should have arbitrated at least one match. This implies that we should find at least 22 dyads containing the given referee (a team has 11 players).
We thus check that for each referee we obtain at least 22 dyads:

In [ ]:
referees = pd.DataFrame(dyads.groupby(['refNum']).size())
referees.columns = ['count']
bad_refs = referees[referees['count'] < 22]
nb_refs = len(referees)
nb_badrefs = len(bad_refs)
print("Total number of refs:", nb_refs)
print("Number of refs with missing data:", nb_badrefs)
print("Percentage of refs with missing data:", 100 * (nb_badrefs / nb_refs), "%")

We found 2149 referees contained in less than 22 dyads. We can consider these as missing data events. 

In [ ]:
dyads_w_rm = dyads.drop(bad_refs.index)
nb_dyads = len(dyads)
nb_dyads_wrm = len(dyads_w_rm)
print("Number of dyads:", nb_dyads)
print("Number of dyads after deletions:", nb_dyads_wrm)
print("Percent of dyads lost:", (100 * (1 - (nb_dyads_wrm / nb_dyads))),"%")

We see that even that the referees to delete represented 68% of the set of referees, the percentage of lost dyads after a potential removal is only around 1.5%. We thus decide to apply the deletion.

In [ ]:
dyads = dyads.drop(bad_refs.index)

### 3. Defining the label

Our goal is to define if a player has dark skin or light skin. We thus need to design a model and train it on a part of this data with predefined skin labels. We will take the columns 'rater1' and 'rater2' to define this label. As the model needs an output (a valid skin label) we need at least one of these columns to be valid.

In [ ]:
dyads = dyads[~(np.isnan(dyads['rater1']) & np.isnan(dyads['rater2']))]
dyads.reset_index(inplace=True)

As we have two observations defining the skin's color for each player, we want to aggregate them but first we have to check if there exists disparities between them. We also check what are the different values given by each rater.

In [ ]:
print("The max difference between two raters is", (dyads['rater1']-dyads['rater2']).max())
print("Rating values given by rater 1", dyads['rater1'].unique())
print("Rating values given by rater 2", dyads['rater2'].unique())

The scale is going from 0 to 1 with a step of 0.25 and the max difference is 0.25.
Therefore we cannot find a case where there are opposite opinions about a player (one black, the other white)

We choose to transform this scale of rating into binary labels. We consider our model to predict more of a white/coloured ouput. We choose to take the mean of these values and if the result is smaller than 0.5 we want to output 0 ('white') and if it is greater than 0.5 output 1 ('black/coloured').

In [ ]:
dyads['y'] = (dyads['rater1'] + dyads['rater2']) / 2 
dyads.loc[dyads['y'] < 0.5, 'y'] = 0
dyads.loc[dyads['y'] >= 0.5, 'y'] = 1
dyads.drop(['rater1', 'rater2'], axis=1, inplace=True)

### 4. Training the classifier - Round one
Now that we have clean the data, we can try to create a classifier on it and train it. We first pop the labels, then before label encoding the object columns in the original dataframe, we fill them first with the string 'Unknown' if we encounter a NaN value.

Once the object columns are label encoded, we still fill the NaN values with -999. This is an arbitrary value we choose, since every column should not contain a negative value, it behaves as if it was not known.

In [ ]:
dyads_encoded = dyads.copy()
dyads_encoded['position'].fillna('Unknown', inplace=True)
dyads_encoded['Alpha_3'].fillna('Unknown', inplace=True)
labels = dyads_encoded.pop('y')
dyads_encoded = preproc_helper.label_encode(dyads_encoded, list(dyads_encoded.select_dtypes(include=['object']).columns))
dyads_encoded.fillna(-999, inplace=True)

In [ ]:
classifier = RandomForestClassifier(n_estimators=10)
scores = cross_val_score(classifier, dyads_encoded, labels, cv=20)
print("Cross-val score:", scores.mean())

The classifier score is impressive. Let's look at the important features.

In [ ]:
#Feature Importances
class_helper.plot_importances(classifier, dyads_encoded, labels, 0.005)

Something is going wrong (why photoID should be an important feature?). Since we train our model with the dyads it means that a same player can be in the train and in the test dataset at the same time, so indeed it's logical that the classifier uses the player name and its unique related informations (birthday, photoID, playerShort) to classify.

As such the model does not make sense.

### 4. Training the classifier - Round two

As seen before the model does not make sense. Hence we move to an approach where we transform our data into a frame with an entry per player. Therefore, this implies, aggregating the dyads players while applying different aggregation functions per columns based on the information it contains. We will thus need to make some choices of interpretation to do the latter.

#### 4.1 Aggregating and cleaning the data

We first aggregate the dyads by players while appending their column informations into arrays. We also need  to not reorder the arrays to keep a mapping between a referee and its informations concerning the player.

In [ ]:
players = dyads.groupby(['playerShort'])
players = preproc_helper.groups_to_lists(players, 'playerShort')

As seen in the important features above, some of the columns are unique and represents directly a player. It means that they won't help to classify them. So we decide to drop them.

In [ ]:
players = players.drop(['birthday', 'player', 'photoID', 'Alpha_3', 'refCountry','index', 'refNum', 'nIAT', 'nExp'], axis=1)

We have a repetition of values for some columns that are supposed to be constant: 
weight, height, club. We check if the respective lists of those columns per player contain the same value or not.

In [ ]:
for column in ['height', 'weight', 'y', 'club', 'leagueCountry']:
    print(column, ':', preproc_helper.has_same_value(players[column]))

As we can see, height and weight can differ for a given player. We check where it's the case and if all the values are NaN for these records (if it's the case it means we cannot deduce a height or a weight based on other observations).

In [ ]:
players_without_height_or_weight = set()
for column in ['height', 'weight']:
    print('\n', column.upper())
    for index, item in players[column].iteritems():
        unique_values = set(item)
        if(len(unique_values) > 1):
            print(index, all(np.isnan(x) for x in unique_values))
            players_without_height_or_weight.add(index)

We hence drop the players that have no datas for their height or their weight.

In [ ]:
players = players.drop(players_without_height_or_weight)

Now they all have the same value so we will represent it by the first element of each list

#### 4.2 Reducing
We apply different aggregation functions on our columns:
- As discussed, the columns  club, leagueCountry, height and weight are constants. We only take the first element of the array. This is also the case for y since a player do not change skin color over time.
- Since the player's position may change over time, we decide to apply a majority vote on the list of his positions. We also create a "Unknown" position class for the Na cases.
- The columns games, victories, ties, defeats, goals, yellowCards, redCards and yellowReds are simply summed.
- seIAT and seExp becomes the standard deviation of the meanIAT and meanExp a player has.
- We take the average of meanIAT and meanExp

In [ ]:
aggregating_functions = [
    (['y'], lambda x: int(x[0])), 
    (['club', 'leagueCountry', 'height', 'weight'], lambda x: x[0]),
    (['position'], mode),
    (['games', 'victories', 'ties', 'defeats', 'goals', 'yellowCards', 'yellowReds', 'redCards'], sum),
    ([('seIAT', 'meanIAT'), ('seExp', 'meanExp')], np.std),
    (['meanIAT', 'meanExp'], np.mean)
]
players = aggr_helper.aggregate_dyads_to_players(players, aggregating_functions)
players.head()

Before training our classifier, we drop the last NaNs and we pop the labels.

In [ ]:
players.dropna(inplace=True)

In [ ]:
labels = players.pop('y')

We encode our categorical data columns. The columns club, leagueCountry and position are supposed to be categorical data. We could dummily encode them with indices as scikit's label_encoding does. However, sklearn's RandomForestClassifier consider those values as continuous. At a given branch, it will separate the obtained values through intervals making the distance between indices important.
Therefore we decide to one-hot encode our data. It slightly increases our feature dimensions but will reduce our bias.

In [ ]:
encoded_player_data = preproc_helper.label_encode(players, list(players.select_dtypes(include=['object']).columns))

We know create our random classifier and train it using a 20-fold cross validation.

In [ ]:
clf = RandomForestClassifier(n_estimators=10)
scores = cross_val_score(clf, encoded_player_data, labels, cv=20)

print("Cross-val score:", scores.mean())

So we got a score around 79% (that makes more sense) but we know that we are losing informations on the cards given by different referees. It means that for a player the value of a card given by a "racist" referee is the same as the value of a card given by a "not racist" referee. We don't want to lose this information, so we will need to ponder the cards using the "racist" value given for a referee.

#### 4.3 Ponderation of cards

First of all we normalize the value of meanIAT and meanEXP in our dyads dataset.

In [ ]:
norm_dyads = dyads.copy()
norm_dyads['meanIATNorm'] = (norm_dyads['meanIAT'] - norm_dyads['meanIAT'].min()) / (norm_dyads['meanIAT'].max() - norm_dyads['meanIAT'].min())
norm_dyads['meanExpNorm'] = (norm_dyads['meanExp'] - norm_dyads['meanExp'].min()) / (norm_dyads['meanExp'].max() - norm_dyads['meanExp'].min())

Once we have that, we create for each dyad a referee score and 3 new columns that ponders the value of the card (yellow/red/yellowRed) given by the referee score for that match.

In [ ]:
norm_dyads['refScore'] = (norm_dyads['meanIATNorm'] + norm_dyads['meanExpNorm']) / 2
norm_dyads['yellowCardsPonder'] = norm_dyads['refScore'] * norm_dyads['yellowCards']
norm_dyads['redCardsPonder'] = norm_dyads['refScore'] * norm_dyads['redCards']
norm_dyads['yellowRedsPonder'] = norm_dyads['refScore'] * norm_dyads['yellowReds']

As before we aggregate this dyad data to get an entry per player. 

In [ ]:
norm_dyads = norm_dyads.groupby(['playerShort'])
norm_dyads = preproc_helper.groups_to_lists(norm_dyads, 'playerShort')

norm_dyads.drop(players_without_height_or_weight, inplace=True)
norm_dyads.drop(['birthday', 'player', 'photoID', 'Alpha_3', 'refCountry', 'nIAT', 'nExp','index', 'refNum', 'meanIATNorm', 'meanExpNorm', 'refScore'], axis=1, inplace=True)

aggregating_functions = [
    (['y'], lambda x: int(x[0])), 
    (['club', 'leagueCountry', 'height', 'weight'], lambda x: x[0]),
    (['position'], mode),
    (['games', 'victories', 'ties', 'defeats', 'goals', 'yellowCards', 'yellowReds', 'redCards', 'yellowCardsPonder', 'yellowRedsPonder', 'redCardsPonder'], sum),
    ([('seIAT', 'meanIAT'), ('seExp', 'meanExp')], np.std),
    (['meanIAT', 'meanExp'], np.mean)
]

players_norm = aggr_helper.aggregate_dyads_to_players(norm_dyads, aggregating_functions)
players_norm.dropna(inplace=True)

players_norm_encoded = preproc_helper.one_hot_encode(players_norm, ['club', 'leagueCountry','position'])

We now pop the labels from the dataframe and we build the model.

In [ ]:
labels_norm = players_norm_encoded.pop('y')

In [ ]:
clf = RandomForestClassifier(n_estimators=10)
scores = cross_val_score(clf, players_norm_encoded, labels_norm, cv=20)
print("Cross-val score:", scores.mean())

### 5. Features Importance Analysis

We now have satisfying data preprocessing and classification. We will look at the influence of the features on our model. In other words, we will measure the feature importances as well as the model behaviour in the case of feature modifications (mainly deletion).

#### 5.1 Features importance with hot encoded labels (club, league country and position)

In [ ]:
class_helper.plot_importances(clf, players_norm_encoded, labels_norm, 0.005)

We remark that the features that often comes at the top of our plot are the two standard deviation of IAT and Exp and also their mean value. Let's now tweak our data by removing/encoding/adding some columns and see if we can get different important features

#### 5.2 Features importance dropping encoded labels (club, league country and position)

In [ ]:
players_norm_no_encoding = players_norm.drop(['club', 'leagueCountry','position'], axis=1)
scores = cross_val_score(clf, players_norm_no_encoding, players_norm_no_encoding.pop('y'), cv=8)
print("Cross-val score:", scores.mean())

In [ ]:
class_helper.plot_importances(clf, players_norm_no_encoding, labels_norm, 0.005)

We will then try to not encode the column using Hot Encoder but just using Label Encoder.

#### 5.3 Features importance using Label Encoder (club, league country and position)

In [ ]:
label_encoded_data_norm = preproc_helper.label_encode(players_norm, ['club', 'leagueCountry','position'])
labels_norm = label_encoded_data_norm.pop('y')

In [ ]:
scores = cross_val_score(clf, label_encoded_data_norm, labels_norm, cv=20)
print("Cross-val score:", scores.mean())

In [ ]:
#Feature Importances
class_helper.plot_importances(clf, label_encoded_data_norm, labels_norm, 0.005)

#### 5.4 Features importance with only the characteristics of the player using Label Encoder
Now we try to build a model where we only have the characteristics of the player itself and not the referee informations.

In [ ]:
label_encoded_data_norm = preproc_helper.label_encode(players_norm, ['club', 'leagueCountry','position'])
player_characteristics = label_encoded_data_norm.drop(['seIAT', 'meanIAT', 'meanExp', 'seExp', 'yellowCardsPonder', 'redCardsPonder', 'yellowRedsPonder'], axis=1)
labels_norm = player_characteristics.pop('y')

clf = RandomForestClassifier(n_estimators=10)
scores = cross_val_score(clf, player_characteristics, labels_norm, cv=20)
print("Cross-val score:", scores.mean())

In [ ]:
class_helper.plot_importances(clf, player_characteristics, labels_norm, 0.005)

The model is less accurate but still surprisingly high. In this case the important features are different. It looks like the classifier uses the results of the matches (victories/ties/defeats) as its best features.

## Bonus

Let's plot the learning curves for the player characteristics and the dataset with all the columns (including the referee country IAT, Exp data)

In [ ]:
#Feature Importances
import plot_learning_curve as plc
(X_train, X_test, y_train, y_test, values, labels) = class_helper.importances(clf, player_characteristics, labels_norm, 0.005)
plc.plot_learning_curve(clf, 'Learning curves on player characteristics (RandomForestClassifier)',X_train, y_train, cv=20).show()

In [ ]:
#Feature Importances
(X_train, X_test, y_train, y_test, values, labels) = class_helper.importances(clf, players_norm_encoded, labels_norm, 0.005)
plc.plot_learning_curve(clf, 'Learning curve on all data (RandomForestClassifier)',X_train, y_train, cv=20).show()

The bias in the first plot 

## Unsupervised Classification

We want to do a clustering of the soccer players into 2 different clusters. We thus need to transform the data into an aggregated form of it grouped by players. As we did the aggregation as well as some cleaning, we will directly reuse the df we obtained after those transformations.
This implies that we use the same reducing functions as well. We also keep the new columns yellowCardsPonder,redCardsPonder and yellowRedsPonder.
Of course, we take a version of the data without the label y containing the skin colour information.

In [63]:
descriptions = players_norm.copy()
gr_truth = labels.copy()
whole = pd.concat([gr_truth,descriptions], axis=1)
descriptions.head()

,club,defeats,games,goals,height,leagueCountry,meanExp,meanIAT,position,redCards,redCardsPonder,seExp,seIAT,ties,victories,weight,yellowCards,yellowCardsPonder,yellowReds,yellowRedsPonder
playerShort,,,,,,,,,,,,,,,,,,,,
aaron-hughes,Fulham FC,222,627,9,182.0,England,0.495149,0.346709,Center Back,0,0.000000,0.231539,0.031525,171,234,71.0,19,7.325574,0,0.000000
aaron-hunt,Werder Bremen,120,334,62,183.0,Germany,0.448310,0.349060,Attacking Midfielder,1,0.336298,0.173776,0.022303,73,141,73.0,42,15.321594,0,0.000000
aaron-lennon,Tottenham Hotspur,112,399,30,165.0,England,0.491392,0.346055,Right Midfielder,0,0.000000,0.225670,0.028481,96,191,63.0,10,3.696365,0,0.000000
aaron-ramsey,Arsenal FC,65,251,37,178.0,England,0.515506,0.346706,Center Midfielder,1,0.335392,0.250735,0.032627,42,144,76.0,31,11.886911,0,0.000000
abdelhamid-el-kaoutari,Montpellier HSC,43,124,1,180.0,France,0.335587,0.331600,Center Back,2,0.670785,0.238918,0.045100,40,41,73.0,8,2.683139,4,1.405477


We will now encode our label columns and then standardize all our features. This will remove some bias in our clustering models (even more so for K-Means for instance). It will equalize our feature variances thus preventing some features to have more weight during computation.
For the sake of model selection, we will encode our our label columns using two different methods mainly one-hot and label encoding and then compare the obtained clusters.

In [64]:
#Dummy Label Encoding
dummy_descr = pprh.label_encode(descriptions.copy(), ['leagueCountry', 'club', 'position'])
dummy_descr = (dummy_descr - dummy_descr.mean()) / dummy_descr.std()

#One-hot Encoding
onehot_descr = pprh.one_hot_encode(descriptions.copy(), ['leagueCountry', 'club', 'position'])
onehot_descr = (onehot_descr - onehot_descr.mean()) / onehot_descr.std()


dummy_descr.head()

NameError: name 'pprh' is not defined

### K-Means

We now computer a first clustering using K-Means with 2 clusters. We will evaluate our clusters using the silhouette score which outputs an score of the clustering quality in the range [-1;1] with 1 being a perfect clustering.

In [65]:
from sklearn.cluster import KMeans 
from sklearn.metrics import silhouette_score

kmeans = KMeans(n_clusters=2, random_state=1)
dummy_labels = kmeans.fit(dummy_descr).labels_
dummy_score = silhouette_score(dummy_descr, dummy_labels)
print("Dummy Encoding: "+str(dummy_score))
loss1 = np.mean(np.absolute(np.subtract(dummy_labels, list(1-gr_truth))))
print("0-1 Loss score:" + str(loss1))

kmeans = KMeans(n_clusters=2, random_state=0)
onehot_labels = kmeans.fit(onehot_descr).labels_
onehot_score = silhouette_score(onehot_descr, onehot_labels)
print("One-Hot Encoding: "+str(onehot_score))
loss1 = np.mean(np.absolute(np.subtract(onehot_labels, list(1-gr_truth))))
print("0-1 Loss score:" + str(loss1))

NameError: name 'dummy_descr' is not defined

We see that the dummy encoding is better than one-hot. The only plausible reason is that our one-hot encoding increases our feature size too much thus increasing our vectors' dimensions. Even if the dummy encoding is not adapted for distance computations, it gains in the low space dimension.

We will now try to visualize our clustering and compare it to the ground truth clustering. In order to visualize to latters, we will apply a PCA to apply a dimensionality reduction to 2 features.

In [66]:
from sklearn.decomposition import PCA

#Feature Transformation
pca = PCA(n_components=2).fit(onehot_descr) 
data_pca = pca.transform(onehot_descr)
df = pd.DataFrame(data_pca, columns=['component1', 'component2'])

#Figure
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(12,5))

#Ground Truth Figure
df_truth = pd.concat([gr_truth.reset_index(), df], axis=1)
df_truth_0 = df_truth[df_truth.y == 0]
df_truth_1 = df_truth[df_truth.y == 1]
df_truth_0.plot(kind="scatter", x='component1', y='component2', color='c', label='White Skin', ax=ax1)
df_truth_1.plot(kind="scatter", x='component1', y='component2', color='m', label='Coloured Skin', ax=ax1)
ax1.set_title('Ground Truth Distribution')

#K-Means clusters
dummy_labels = pd.DataFrame(dummy_labels)
dummy_labels.columns = ['y']
df_cl = pd.concat([dummy_labels, df], axis=1)
df_cl_0 = df_cl[df_cl.y == 1]
df_cl_1 = df_cl[df_cl.y == 0]
df_cl_0.plot(kind="scatter", x='component1', y='component2', color='c', label='White Skin', ax=ax2)
df_cl_1.plot(kind="scatter", x='component1', y='component2', color='m', label='Coloured Skin', ax=ax2)
ax2.set_title('K-Means classification')


plt.show()

NameError: name 'onehot_descr' is not defined

Since we are using K-means...BLABLABLA
### Feature removal analysis
TODO: choisir dummy ou onehot!!!

In [68]:
temp_feats = dummy_descr.copy()
cols = temp_feats.columns
end = len(cols)
rm_scores = {}
for i in range(1,end):
    temp = temp_feats[cols[:i]]
    kmm = KMeans(n_clusters=2, random_state=1).fit(temp)
    labels = kmm.labels_
    rm_scores[end-i] = silhouette_score(temp,labels)
print(rm_scores.items())

NameError: name 'dummy_descr' is not defined

In [ ]:
We now plot the silhouette score against the number of features taken into account.